In [8]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
import matplotlib.pyplot as plt
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as soup

%matplotlib inline  

In [9]:
def cleanString(a):
    b = a.replace('\t','')
    c = b.replace('\r','')
    d = c.replace('\n','')
    return d

In [10]:
def webScrapeTable(url, xPathy, rowLength):
    ###url = "https://www.atptour.com/en/rankings/singles"
    ###!!!!!!!!!!!! Inspect the webpage and xpath should be like this '//*[@id="pageEventH2hTable"]/table/tbody/tr'
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath(xPathy)
    ##
    ##
    ####Check the length of the first 12 rows
    ###[len(T) for T in tr_elements[:12]]
    #Create empty list
    col=[]
    i=0
    #For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        name = cleanString(name)
        #print('%d:"%s"' % (i,name))
        col.append((name,[]))
    ##
    ##
    #Since out first row is the header, data is stored on the second row onwards
    for j in range(1,len(tr_elements)):
        #T is our j'th row
        T=tr_elements[j]
        
        i = 0
        
        #If row is not of size rowLength, the //tr data is not from our table 
        if len(T)!=rowLength:
            if len(T) == rowLength-1:
                i = 1
                col[0][1].append(0)
            else:
                continue
                #break
    
        ##i is the index of our column
        #i=0
    
        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            data = cleanString(data)
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            
            col[i][1].append(data)
            #Increment i for the next column
            i+=1
    ##
    ##
    #[len(C) for (title,C) in col] Use this to check that each column is the same length
    Dict={title:column for (title,column) in col}
    df=pd.DataFrame(Dict)
    df.head()
    return df

In [11]:
#Function to run when the dataframe didn't have a header so the header is supposed to be the first row of data
def headToData(df):
    headList = []
    for i in range(len(df.columns)):
        value = str(df.columns[i])
        headList.append(value)
    df.loc[-1] = headList
    df.index = df.index+1
    df = df.sort_index()
    return df
#Remember that you still need to rename the columns of the dataframe
#You have to do something like this
#ThiemZverevdf = headToData(ThiemZverevFunctionCheck)
#ThiemZverevdf.columns = ['Year', 'Tournament', 'Surface', 'Round', 'Winner', 'Score']
#ThiemZverevdf.head()

In [12]:
def h2hpercent(df):
    newdf = pd.get_dummies(df['Winner'])
    a = newdf.shape
    aplayer = 0
    bplayer = 0
    aplayerName = ''
    if a[1] == 1:
        aplayer = a[0]
        aplayerName = str(newdf.columns[0])
        return [aplayer, aplayerName, 'Won all', a[0]]
    else:
        for i in range(a[0]):
            aplayer = aplayer + newdf.iloc[i,0]
            bplayer = bplayer + newdf.iloc[i,1]
        tot = aplayer + bplayer
        apercent = aplayer / tot
        bpercent = bplayer / tot
        return [apercent, bpercent, tot]
#Important note, this spits out player stats based on alphabetical order of first name

In [13]:
#Data is in format 0-0, so I'll need to write a function to split this into wins and losses separately
def winloss(A):
    hyphenLoc = A.index('–')
    wins = A[:hyphenLoc]
    losses = A[hyphenLoc+1:]
    return [wins, losses]

In [14]:
#Now we want to scale the wins and losses for relevance based on recency
def scaleWins(dfName, dataframeRowName, fromYear, toYear):
    rowdf = dfName.loc[[dataframeRowName]]
    index1 = rowdf.columns.get_loc(fromYear)
    index2 = rowdf.columns.get_loc(toYear)
    scaleList = list(range(1,(index2-index1+2)))
    scaledWins = 0
    scaledLosses = 0
    for i in range(index1, index2+1):
        rowdfString = rowdf.iloc[0,i]
        winlossList = winloss(rowdfString)
        scaledWins = scaledWins + scaleList[i]*int(winlossList[0])
        scaledLosses = scaledLosses + scaleList[i]*int(winlossList[1])
    return [scaledWins, scaledLosses]

In [ ]:
#Lets make dictionaries of tournaments that have happened so far this year
TournamentNameDict = {
    "Qatar": "https://en.wikipedia.org/wiki/List_of_Australian_Open_men%27s_singles_champions",
    "Adelaide": "https://en.wikipedia.org/wiki/Indian_Wells_Masters",
    "ASB": "https://en.wikipedia.org/wiki/Miami_Open_(tennis)",
    "Australia": "https://en.wikipedia.org/wiki/Monte-Carlo_Masters",
    "Cordoba": "https://en.wikipedia.org/wiki/Madrid_Open_(tennis)",
    "Tata": "https://en.wikipedia.org/wiki/Italian_Open_(tennis)",
    "Open Sud": "https://en.wikipedia.org/wiki/List_of_French_Open_men%27s_singles_champions",
    "ABN": "https://en.wikipedia.org/wiki/List_of_Wimbledon_gentlemen%27s_singles_champions",
    "NY Open": "https://en.wikipedia.org/wiki/Canadian_Open_(tennis)",
    "Argentina": "https://en.wikipedia.org/wiki/Cincinnati_Masters",
    "Rio": "https://en.wikipedia.org/wiki/List_of_US_Open_men%27s_singles_champions",
    "Open 13": "https://en.wikipedia.org/wiki/Shanghai_Masters_(tennis)",
    "Delray": "https://en.wikipedia.org/wiki/Paris_Masters", 
    "Dubai": "",
    "Abierto": "",
    "Chile":""
}

#Lets also make a dictionary of XPaths for the tables we are interested in: Winners and Runners Up
TournamentWinsDict = {
    "Australia": '//*[@id="mw-content-text"]/div/table[4]/tbody/tr',
    "Indian Wells": '//*[@id="mw-content-text"]/div/table[3]/tbody/tr',
    "Miami": '//*[@id="mw-content-text"]/div/table[5]/tbody/tr',
    "Monte Carlo": '//*[@id="mw-content-text"]/div/table[2]/tbody/tr',
    "Madrid": '//*[@id="mw-content-text"]/div/table[2]/tbody/tr',
    "Rome": '//*[@id="mw-content-text"]/div/table[3]/tbody/tr',
    "French": '//*[@id="mw-content-text"]/div/table[4]/tbody/tr',
    "Wimbledon": '//*[@id="mw-content-text"]/div/table[4]/tbody/tr',
    "Rogers Cup": '//*[@id="mw-content-text"]/div/table[3]/tbody/tr',
    "Cincinnati": '//*[@id="mw-content-text"]/div/table[3]/tbody/tr',
    "US": '//*[@id="mw-content-text"]/div/table[4]/tbody/tr',
    "Shanghai": '//*[@id="mw-content-text"]/div/table[4]/tbody/tr',
    "Paris": '//*[@id="mw-content-text"]/div/table[2]/tbody/tr',
}

In [ ]:
def webScrapeTable(TournName): #url, xPathy, rowLength):
    url = TournamentNameDict[TournName]
    xPathy = TournamentWinsDict[TournName]
    ###!!!!!!!!!!!! Inspect the webpage and xpath should be like this '//*[@id="pageEventH2hTable"]/table/tbody/tr'
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath(xPathy)
    ##
    ##
    ####Check the length of the first 12 rows
    rowLength = [len(T) for T in tr_elements[0]]
    #rowLength = rowLength[0]
    #print(rowLength)
    rowLength = len(rowLength)
    ###[len(T) for T in tr_elements[:12]]
    #Create empty list
    col=[]
    i=0
    #For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        name = cleanString(name)
        #print('%d:"%s"' % (i,name))
        col.append((name,[]))
    ##
    ##
    #Since out first row is the header, data is stored on the second row onwards
    for j in range(1,len(tr_elements)):
        #T is our j'th row
        T=tr_elements[j]
        
        i = 0
        
        #If row is not of size rowLength, the //tr data is not from our table 
        if len(T)!=rowLength:
            if len(T) == rowLength-1:
                i = 1
                col[0][1].append(0)
            else:
                continue
                #break
    
        ##i is the index of our column
        #i=0
    
        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            data = cleanString(data)
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            
            col[i][1].append(data)
            #Increment i for the next column
            i+=1
    ##
    ##
    #[len(C) for (title,C) in col] Use this to check that each column is the same length
    Dict={title:column for (title,column) in col}
    df=pd.DataFrame(Dict)
    df.head()
    return df

In [178]:
#Lets make dictionaries of tournaments that have happened so far this year
TournamentNameDict = {
    "Qatar": "https://www.atptour.com/en/scores/archive/doha/451/2020/results",
    "Adelaide": "https://www.atptour.com/en/scores/archive/adelaide/8998/2020/results",
    "ASB": "https://www.atptour.com/en/scores/archive/auckland/301/2020/results",
    "Australia": "https://www.atptour.com/en/scores/archive/australian-open/580/2020/results",
    "Cordoba": "https://www.atptour.com/en/scores/archive/cordoba/9158/2020/results",
    "Tata": "https://www.atptour.com/en/scores/archive/pune/891/2020/results",
    "Open Sud": "https://www.atptour.com/en/scores/archive/montpellier/375/2020/results",
    "ABN": "https://www.atptour.com/en/scores/archive/rotterdam/407/2020/results",
    "NY Open": "https://www.atptour.com/en/scores/archive/new-york/424/2020/results",
    "Argentina": "https://www.atptour.com/en/scores/archive/buenos-aires/506/2020/results",
    "Rio": "https://www.atptour.com/en/scores/archive/rio-de-janeiro/6932/2020/results",
    "Open 13": "https://www.atptour.com/en/scores/archive/marseille/496/2020/results",
    "Delray": "https://www.atptour.com/en/scores/archive/delray-beach/499/2020/results", 
    "Dubai": "https://www.atptour.com/en/scores/archive/dubai/495/2020/results",
    "Abierto": "https://www.atptour.com/en/scores/archive/acapulco/807/2020/results",
    "Chile":"https://www.atptour.com/en/scores/archive/santiago/8996/2020/results"
}

#Lets also make a dictionary of XPaths for the tables we are interested in: First Round
TournamentWinsDict = {
    "Qatar": '//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr',
    "Adelaide": '//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr',
    "ASB": '//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr',
    "Australia": '//*[@id="scoresResultsContent"]/div/table/tbody[7]/tr',
    "Cordoba": '//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr',
    "Tata": '//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr',
    "Open Sud": '//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr',
    "ABN": '//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr',
    "NY Open": '//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr',
    "Argentina": '//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr',
    "Rio": '//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr',
    "Open 13": '//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr',
    "Delray": '//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr',
    "Dubai":'//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr',
    "Abierto":'//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr',
    "Chile":'//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr'
}

In [74]:
url = "https://www.atptour.com/en/scores/archive/santiago/8996/2020/results"
pathy = '//*[@id="scoresResultsContent"]/div/table/tbody[5]/tr'
rowLen = 10
testdf = webScrapeTable(url, pathy, rowLen)
Tourndf = headToData(testdf)
Tourndf.columns = ['BS', 'P1 Seeding', 'P1 Name', 'Defeats', 'P2 Seeding', 'P2 Name', 'Score', 'BS2', 'BS3']
Tourndf = Tourndf[['P1 Name', 'P2 Name']]
Tourndf.head()

,P1 Name,P2 Name
0,Juan Ignac...,Federico C...
1,Hugo Delli...,Marco Cecc...
2,Federico D...,Martin Kli...
3,Thiago Mon...,Leonardo M...
4,Marcelo To...,Carlos Tab...


In [ ]:
playSet = list(set().union(WinNames, FinalNames))

In [92]:
playSet = []
namesList = ['ABN','Dubai', 'Abierto'
            ]
for i in range(len(namesList)):
    name = namesList[i]
    url = TournamentNameDict[name]
    pathy = TournamentWinsDict[name]
    rowLen = 10
    testdf = webScrapeTable(url, pathy, rowLen)
    Tourndf = headToData(testdf)
    Tourndf.columns = ['BS', 'P1 Seeding', 'P1 Name', 'Defeats', 'P2 Seeding', 'P2 Name', 'Score', 'BS2', 'BS3']
    Tourndf = Tourndf[['P1 Name', 'P2 Name']]
    WinNames = Tourndf["P1 Name"].values
    FinalNames = Tourndf["P2 Name"].values
    playSet = set().union(playSet, WinNames, FinalNames)
    

In [149]:
#playList = list(playSet)
#for i in range(len(playList)):
#    playList[i] = playList[i].strip()
#len(playList)
playList[1]

'Nikoloz Basilashvili'

In [151]:
playDict = {
    playList[0]:"https://www.atptour.com/en/players/grigor-dimitrov/d875/overview",
    playList[1]:"https://www.atptour.com/en/players/nikoloz-basilashvili/bg23/overview",
    playList[2]:"https://www.atptour.com/en/players/malek-jaziri/j267/overview",
    playList[3]:"https://www.atptour.com/en/players/yen-hsun-lu/l575/overview",
    playList[4]:"https://www.atptour.com/en/players/frances-tiafoe/td51/overview",
    playList[5]:"https://www.atptour.com/en/players/alex-de-minaur/dh58/overview",
    playList[6]:"https://www.atptour.com/en/players/mohamed-safwat/sk47/overview",
    playList[7]:"https://www.atptour.com/en/players/felix-auger-aliassime/ag37/overview",
    playList[8]:"https://www.atptour.com/en/players/john-millman/mh30/overview",
    playList[9]:"https://www.atptour.com/en/players/daniel-evans/e687/overview",
    playList[10]:"https://www.atptour.com/en/players/daniil-medvedev/mm58/overview",
    playList[11]:"https://www.atptour.com/en/players/denis-shapovalov/su55/overview",
    playList[12]:"https://www.atptour.com/en/players/jason-jung/j380/overview",
    playList[13]:"https://www.atptour.com/en/players/pierre-hugues-herbert/h996/overview",
    playList[14]:"https://www.atptour.com/en/players/jannik-sinner/s0ag/overview",
    playList[15]:"https://www.atptour.com/en/players/tommy-paul/pl56/overview",
    playList[16]:"https://www.atptour.com/en/players/robin-haase/h756/overview",
    playList[17]:"https://www.atptour.com/en/players/dusan-lajovic/l987/overview",
    playList[18]:"https://www.atptour.com/en/players/richard-gasquet/g628/overview",
    playList[19]:"https://www.atptour.com/en/players/karen-khachanov/ke29/overview",
    playList[20]:"https://www.atptour.com/en/players/ricardas-berankis/be90/overview",
    playList[21]:"https://www.atptour.com/en/players/vasek-pospisil/pd07/overview",
    playList[22]:"https://www.atptour.com/en/players/marin-cilic/c977/overview",
    playList[23]:"https://www.atptour.com/en/players/aljaz-bedene/bh09/overview",
    playList[24]:"https://www.atptour.com/en/players/alexander-bublik/bk92/overview",
    playList[25]:"https://www.atptour.com/en/players/lorenzo-musetti/m0ej/overview",
    playList[26]:"https://www.atptour.com/en/players/novak-djokovic/d643/overview",
    playList[27]:"https://www.atptour.com/en/players/gilles-simon/sd32/overview",
    playList[28]:"https://www.atptour.com/en/players/john-isner/i186/overview",
    playList[29]:"https://www.atptour.com/en/players/radu-albot/a829/overview",
    playList[30]:"https://www.atptour.com/en/players/feliciano-lopez/l397/overview",
    playList[31]:"https://www.atptour.com/en/players/stefanos-tsitsipas/te51/overview",
    playList[32]:"https://www.atptour.com/en/players/mackenzie-mcdonald/mk66/overview",
    playList[33]:"https://www.atptour.com/en/players/miomir-kecmanovic/ki95/overview",
    playList[34]:"https://www.atptour.com/en/players/hubert-hurkacz/hb71/overview",
    playList[35]:"https://www.atptour.com/en/players/roberto-bautista-agut/bd06/overview",
    playList[36]:"https://www.atptour.com/en/players/mischa-zverev/z168/overview",
    playList[37]:"https://www.atptour.com/en/players/gael-monfils/mc65/overview",
    playList[38]:"https://www.atptour.com/en/players/nick-kyrgios/ke17/overview",
    playList[39]:"https://www.atptour.com/en/players/jan-lennard-struff/sl28/overview",
    playList[40]:"https://www.atptour.com/en/players/benoit-paire/pd31/overview",
    playList[41]:"https://www.atptour.com/en/players/yoshihito-nishioka/n732/overview",
    playList[42]:"https://www.atptour.com/en/players/gregoire-barrere/bk24/overview",
    playList[43]:"https://www.atptour.com/en/players/marcos-giron/gc88/overview",
    playList[44]:"https://www.atptour.com/en/players/taro-daniel/da81/overview",
    playList[45]:"https://www.atptour.com/en/players/philipp-kohlschreiber/k435/overview",
    playList[46]:"https://www.atptour.com/en/players/prajnesh-gunneswaran/ga94/overview",
    playList[47]:"https://www.atptour.com/en/players/ugo-humbert/hh26/overview",
    playList[48]:"https://www.atptour.com/en/players/andrey-rublev/re44/overview",
    playList[49]:"https://www.atptour.com/en/players/adrian-mannarino/me82/overview",
    playList[50]:"https://www.atptour.com/en/players/kyle-edmund/e831/overview",
    playList[51]:"https://www.atptour.com/en/players/filip-krajinovic/kb05/overview",
    playList[52]:"https://www.atptour.com/en/players/marton-fucsovics/f724/overview",
    playList[53]:"https://www.atptour.com/en/players/soonwoo-kwon/kf17/overview",
    playList[54]:"https://www.atptour.com/en/players/yasutaka-uchiyama/u134/overview",
    playList[55]:"https://www.atptour.com/en/players/rafael-nadal/n409/overview",
    playList[56]:"https://www.atptour.com/en/players/pedro-martinez/mo44/overview",
    playList[57]:"https://www.atptour.com/en/players/dennis-novak/db59/overview",
    playList[58]:"https://www.atptour.com/en/players/damir-dzumhur/d923/overview",
    playList[59]:"https://www.atptour.com/en/players/gerardo-lopez-villasenor/lg87/overview",
    playList[60]:"https://www.atptour.com/en/players/taylor-fritz/fb98/overview",
    playList[61]:"https://www.atptour.com/en/players/joao-sousa/sh90/overview",
    playList[62]:"https://www.atptour.com/en/players/pablo-andujar/a596/overview",
    playList[63]:"https://www.atptour.com/en/players/alexander-zverev/z355/overview",
    playList[64]:"https://www.atptour.com/en/players/tallon-griekspoor/gj37/overview",
    playList[65]:"https://www.atptour.com/en/players/pablo-carreno-busta/cd85/overview",
    playList[66]:"https://www.atptour.com/en/players/steve-johnson/j386/overview",
    playList[67]:"https://www.atptour.com/en/players/david-goffin/gb88/overview",
    playList[68]:"https://www.atptour.com/en/players/stan-wawrinka/w367/overview",
    playList[69]:"https://www.atptour.com/en/players/cameron-norrie/n771/overview",
    playList[70]:"https://www.atptour.com/en/players/mikhail-kukushkin/k926/overview",
    playList[71]:"https://www.atptour.com/en/players/alex-bolt/bi81/overview",
    playList[72]:"https://www.atptour.com/en/players/lloyd-harris/hg86/overview",
    playList[73]:"https://www.atptour.com/en/players/fabio-fognini/f510/overview",
}

In [161]:
playerStatdf = pd.DataFrame(columns=['Name', 'YTDTitles',"CareerTitles", 'CurrentRank', 'MaxRank', 'YTDWins', 'CareerWins'])
i = 0
for name in playList:
    url = playDict[name]
    pathy = '//*[@id="playersStatsTable"]/tbody/tr[1]'
    pathy2 = '//*[@id="playersStatsTable"]/tbody/tr[2]'
    rowLen = 0
    df = webScrapeTable(url, pathy, rowLen)
    newRow = [name, df.columns[5], df.columns[2], df.columns[4]]
    df = webScrapeTable(url, pathy2, rowLen)
    newRow.insert(2, df.columns[3])
    newRow.insert(4, df.columns[1])
    newRow.insert(6, df.columns[2])
    #newdf = pd.DataFrame(newRow, columns = ['Name', 'YTDTitles',"CareerTitles", 'CurrentRank', 'MaxRank', 'YTDWins', 'CareerWins'], index = [i])
    #newdf = pd.DataFrame(newRow, columns = ['Name', 'YTDTitles',"CareerTitles", 'CurrentRank', 'MaxRank', 'YTDWins', 'CareerWins'], index=['a'])
    #playerStatdf.append()
    playerStatdf.loc[name] = newRow
    #playerStatdf.append(pd.Series(newRow, index = playerStatdf.columns), ignore_index=True)
    #i+=1
    #dfObj.append(pd.Series(['Raju', 21, 'Bangalore', 'India'], index=dfObj.columns ), ignore_index=True)
    
    

In [176]:
#74 rows, 7 columns
for i in range(74):
    for j in range(7):
        playerStatdf.iloc[i][j] = str(playerStatdf.iloc[i][j]).strip()
        if j == 1:
            Tloc = str(playerStatdf.iloc[i][j]).index('T')
            playerStatdf.iloc[i][j] = str(playerStatdf.iloc[i][j])[:Tloc]
        if j == 2:
            Tloc = str(playerStatdf.iloc[i][j]).index('T')
            playerStatdf.iloc[i][j] = str(playerStatdf.iloc[i][j])[:Tloc]
        if j == 3:
            Rloc = str(playerStatdf.iloc[i][j]).index('R')
            playerStatdf.iloc[i][j] = str(playerStatdf.iloc[i][j])[:Rloc]
        if j == 4:
            Cloc = str(playerStatdf.iloc[i][j]).index('C')
            playerStatdf.iloc[i][j] = str(playerStatdf.iloc[i][j])[:Cloc]
        if j == 5:
            hyloc = str(playerStatdf.iloc[i][j]).index('-')
            playerStatdf.iloc[i][j] = str(playerStatdf.iloc[i][j])[:hyloc]
        if j == 6:
            hyloc = str(playerStatdf.iloc[i][j]).index('-')
            playerStatdf.iloc[i][j] = str(playerStatdf.iloc[i][j])[:hyloc]

In [177]:
playerStatdf.head()

,Name,YTDTitles,CareerTitles,CurrentRank,MaxRank,YTDWins,CareerWins
Grigor Dimitrov,Grigor Dimitrov,0,8,19,3,7,311
Nikoloz Basilashvili,Nikoloz Basilashvili,0,3,27,16,3,97
Malek Jaziri,Malek Jaziri,0,0,259,42,0,97
Yen-Hsun Lu,Yen-Hsun Lu,0,0,966,33,0,161
Frances Tiafoe,Frances Tiafoe,0,1,81,29,2,62


In [204]:
TournamentNameDict
TournamentWinsDict
Matchesdf = pd.DataFrame(columns = ['P1 Name', 'P2 Name'])
print(Matchesdf.shape)
numb = 5
pathy = '//*[@id="scoresResultsContent"]/div/table/tbody['+str(numb)+']/tr'
for name in ['ABN','Dubai', 'Abierto']:
    url = TournamentNameDict[name]
    for j in range(5,0,-1):
        rowLen = 10
        pathy = '//*[@id="scoresResultsContent"]/div/table/tbody[' + str(j)+ ']/tr'
        testdf = webScrapeTable(url, pathy, rowLen)
        Tourndf = headToData(testdf)
        Tourndf.columns = ['BS', 'P1 Seeding', 'P1 Name', 'Defeats', 'P2 Seeding', 'P2 Name', 'Score', 'BS2', 'BS3']
        Tourndf = Tourndf[['P1 Name', 'P2 Name']]
        #print(Tourndf.head())
        shapeTup = Tourndf.shape
        for i in range(shapeTup[0]):
            Matchesdf = Matchesdf.append(Tourndf.iloc[i], ignore_index=True)
    #Matchesdf.iloc[0] = Tourndf.iloc[1]

(0, 2)


In [223]:
dfDim = Matchesdf.shape
testdropdf = playerStatdf.drop(['Name'], axis=1)
statsdf = pd.DataFrame(columns = ['YTDTitles',"CareerTitles", 'CurrentRank', 'MaxRank', 'YTDWins', 'CareerWins', 'Win'])
for i in range(dfDim[0]):
    if i%2 == 0:
        P1 = str(Matchesdf.iloc[i][0]) #P1 wins
        P2 = str(Matchesdf.iloc[i][1])
        P1 = P1.strip()
        P2 = P2.strip()
        newRow = []
        k = 0
        for j in testdropdf.columns:
            newRow.append(int(testdropdf.loc[P1][j]) - int(testdropdf.loc[P2][j]))
            k+=1
        newRow.append(1)
        statsdf.loc[i] = newRow
    elif i%2 == 1:
        P1 = str(Matchesdf.iloc[i][1]) #P1 loses
        P2 = str(Matchesdf.iloc[i][0])
        P1 = P1.strip()
        P2 = P2.strip()
        newRow = []
        k = 0
        for j in testdropdf.columns:
            newRow.append(int(testdropdf.loc[P1][j]) - int(testdropdf.loc[P2][j]))
            k+=1
        newRow.append(0)
        statsdf.loc[i] = newRow

In [224]:
statsdf.head()

,YTDTitles,CareerTitles,CurrentRank,MaxRank,YTDWins,CareerWins,Win
0,0,-7,88,21,0,-26,1
1,-1,-4,23,23,-3,-74,0
2,2,7,-56,-22,16,296,1
3,0,-2,159,26,-8,-45,0
4,0,-5,4,-1,5,-234,1


In [226]:
import pylab as pl
import scipy.optimize as opt
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [240]:
X = np.asarray(statsdf[['YTDTitles', 'CareerTitles', 'CurrentRank', 'MaxRank', 'YTDWins', 'CareerWins']])
X=X.astype('int')
X[0:5]

array([[   0,   -7,   88,   21,    0,  -26],
       [  -1,   -4,   23,   23,   -3,  -74],
       [   2,    7,  -56,  -22,   16,  296],
       [   0,   -2,  159,   26,   -8,  -45],
       [   0,   -5,    4,   -1,    5, -234]])

In [239]:
y = np.asarray(statsdf['Win'])
y=y.astype('int')
y [0:5]

array([1, 0, 1, 0, 1])

In [241]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[-0.10174852, -0.31522552,  0.76578247,  0.37785751, -0.02778973,
        -0.14200576],
       [-1.15314986, -0.20174433,  0.23740043,  0.40907986, -0.48386817,
        -0.28756574],
       [ 2.00105417,  0.21435335, -0.40478697, -0.29342289,  2.40462864,
         0.83445912],
       [-0.10174852, -0.12609021,  1.34293823,  0.45591338, -1.24399891,
        -0.19962326],
       [-0.10174852, -0.23957139,  0.0829503 ,  0.03441172,  0.73234101,
        -0.77276569]])

In [242]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (74, 6) (74,)
Test set: (19, 6) (19,)


In [243]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='saga').fit(X_train,y_train)
LR

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [244]:
yhat = LR.predict(X_test)
yhat

array([0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1])

In [245]:
yhat_prob = LR.predict_proba(X_test)
yhat_prob

array([[0.63774907, 0.36225093],
       [0.50761225, 0.49238775],
       [0.45335537, 0.54664463],
       [0.45557668, 0.54442332],
       [0.50131068, 0.49868932],
       [0.50199032, 0.49800968],
       [0.73687462, 0.26312538],
       [0.47059764, 0.52940236],
       [0.60225118, 0.39774882],
       [0.42850909, 0.57149091],
       [0.56526352, 0.43473648],
       [0.52736984, 0.47263016],
       [0.46951408, 0.53048592],
       [0.45143502, 0.54856498],
       [0.54229733, 0.45770267],
       [0.3915329 , 0.6084671 ],
       [0.38475345, 0.61524655],
       [0.31353493, 0.68646507],
       [0.46489411, 0.53510589]])

In [246]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test, yhat)

C:\Users\Tyler\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


0.7894736842105263

In [247]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
print(confusion_matrix(y_test, yhat, labels=[1,0]))

[[8 2]
 [2 7]]


In [46]:
class Player:
    def __init__(self, name, age, country, hard, clay, grass, twolevel, fivelevel, thoulevel, grandslam):
        self.name = name
        self.age = age
        self.country = country
        self.hard = hard
        self.clay = clay
        self.grass = grass
        self.twolevel = twolevel
        self.fivelevel = fivelevel
        self.thoulevel = thoulevel
        self.grandslam = grandslam

In [47]:
Guy = Player('name', 21, 'usa')

TypeError: __init__() missing 7 required positional arguments: 'hard', 'clay', 'grass', 'twolevel', 'fivelevel', 'thoulevel', and 'grandslam'